In [4]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from datetime import datetime, timedelta
import time


In [5]:
# load the API key
load_dotenv()
av_api_key = os.getenv("AV_API_KEY") 

In [6]:
# create a datetime conversion function
def convert_to_datetime(time_stamp):
    date_format = "%Y%m%dT%H%M%S"
    return datetime.strptime(time_stamp, date_format)



In [21]:
# Pull in the list of companies

df_SP_500 = pd.read_csv("constituents.csv")
ticker_list = df_SP_500['Symbol'].astype(str).tolist()
ticker_list = [t.replace('.', '-') for t in ticker_list]

for ticker in ticker_list:
    print(ticker)

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BRK-B
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BSX
BMY
AVGO
BR
BRO
BF-B
BLDR
BG
BXP
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CRWD
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GDDY
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ

In [22]:
# pull the sentiment analysis data for the S&P 500
counter = 0

# define the query date range
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 1, 31)

# ticker_list = ["CE"]

for ticker in ticker_list:
    # create a data frame
    df_tickers = pd.DataFrame()

    # Run a counter & time sleep to pause every 75 pulls
    counter = counter + 1
    counter_check = counter % 75
    # print(counter)

    if counter_check == 0:  
        time.sleep(60)

    pull_start_ann = start_date.strftime("%Y%m%dT%H%M")
    pull_end_ann = end_date.strftime("%Y%m%dT%H%M")


    try:
        # Build the URL and query the API
        url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=' + ticker + '&apikey=M9BTYXZI7EG8P00S' + '&time_from=' + pull_start_ann + '&time_to=' + pull_end_ann + '&limit=1000'
        r = requests.get(url)
        data = r.json()

        # Create the data frame
        df_data = pd.json_normalize(data['feed'])

        if len(df_data) < 500:
            # expand the ticker sentiment column, filter to only the desired ticker, drop unnecessary columns, convert datetime
            try:
                df_data = df_data.explode('ticker_sentiment')
                df_data = df_data[df_data['ticker_sentiment'].apply(lambda x: x['ticker'] == ticker)]
                df_data = pd.concat([df_data.drop(['ticker_sentiment'], axis=1), df_data['ticker_sentiment'].apply(pd.Series)], axis=1)
                df_data = df_data.drop(['banner_image', 'source', 'authors', 'category_within_source', 'topics', 'overall_sentiment_score', 'overall_sentiment_label', 'source_domain'], axis=1)
                df_data['time_published'] = df_data['time_published'].apply(convert_to_datetime)
                df_tickers = pd.concat([df_tickers, df_data], ignore_index=True)
                        
            except KeyError as e:
                print(f"KeyError: {e} - Data not found in DataFrame for {ticker}")

        else:
            # Iterate through each date
            pull_date = start_date
            while pull_date <= end_date:
                # Run a counter & time sleep to pause every 75 pulls
                counter = counter + 1
                counter_check = counter % 75
                # print(counter)

                if counter_check == 0:  
                    time.sleep(60)
                
                # Grab the review period
                pull_start = pull_date.strftime("%Y%m%dT%H%M")
                pull_end_dt = pull_date + timedelta(days=10)
                pull_end = pull_end_dt.strftime("%Y%m%dT%H%M")
                
                # Build the URL and query the API
                url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=' + ticker + '&apikey=M9BTYXZI7EG8P00S' + '&time_from=' + pull_start + '&time_to=' + pull_end + '&limit=1000'
                r = requests.get(url)
                data = r.json()

                # Create the data frame
                df_data = pd.json_normalize(data['feed'])

                # expand the ticker sentiment column, filter to only the desired ticker, drop unnecessary columns, convert datetime
                try:
                    df_data = df_data.explode('ticker_sentiment')
                    df_data = df_data[df_data['ticker_sentiment'].apply(lambda x: x['ticker'] == ticker)]
                    df_data = pd.concat([df_data.drop(['ticker_sentiment'], axis=1), df_data['ticker_sentiment'].apply(pd.Series)], axis=1)
                    df_data = df_data.drop(['banner_image', 'source', 'authors', 'category_within_source', 'topics', 'overall_sentiment_score', 'overall_sentiment_label', 'source_domain'], axis=1)
                    df_data['time_published'] = df_data['time_published'].apply(convert_to_datetime)
                    df_tickers = pd.concat([df_tickers, df_data], ignore_index=True)
                            
                except KeyError as e:
                    print(f"KeyError: {e} - Data not found in DataFrame for {ticker} for {pull_date}")
                    
                # Concat to the main data frame
            
                pull_date += timedelta(days=10)

        # Create an export location and export the data frame to a CSV
        try:
            file_path = 'Media_Sentiment_Scores/' + ticker + '_Sentiments.csv'
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            df_tickers.to_csv(file_path, index=False)

            print(f"DataFrame exported to {file_path}")

        except:
            print(f"CSV not loaded for {ticker}")

    except:
        print(f"No data for {ticker}")

DataFrame exported to Media_Sentiment_Scores/MMM_Sentiments.csv
KeyError: 'ticker_sentiment' - Data not found in DataFrame for AOS
DataFrame exported to Media_Sentiment_Scores/AOS_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/ABT_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/ABBV_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/ACN_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/ADBE_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/AMD_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/AES_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/AFL_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/A_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/APD_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/ABNB_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/AKAM_Sentiments.csv
DataFrame exported to Media_Sentiment_Scores/ALB_Sentiments.csv
DataFrame exported to Media_Sentime